# Persistent homology of activations

In this notebook we will show how the layers of a neural network trained on a classifiction task transforms the input data cloud.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [6]:
from gdeep.neural_nets import sample_nn

ModuleNotFoundError: No module named 'gdeep'